## Data proccessing for supplementary models

In [1]:
import os
import pandas as pd
os.chdir('/compass/Shared/Users/bogdanov/vyzkumny_ukol')
import process_data_for_alt_models as pda
shot_usage = pd.read_csv(f'./data/shot_usage.csv')
shot_for_alt = shot_usage[shot_usage['used_for_alt']]
shot_numbers = shot_for_alt['shot']



In [2]:
pda.process_data_for_alt_models(shot_numbers, variant='seidl_2023', sampling_freq=300)

pda.process_data_for_alt_models(shot_numbers, variant='seidl_2023', sampling_freq=150)

pda.process_data_for_alt_models(shot_numbers, variant='seidl_2023', sampling_freq=30)

  0%|          | 0/53 [00:00<?, ?it/s]

working on shot: 13182


  2%|▏         | 1/53 [00:02<01:59,  2.29s/it]

working on shot: 16532


  4%|▍         | 2/53 [00:03<01:34,  1.86s/it]

...

 96%|█████████▌| 51/53 [00:30<00:01,  1.63it/s]

working on shot: 20146


 98%|█████████▊| 52/53 [00:31<00:00,  1.66it/s]

working on shot: 20147


100%|██████████| 53/53 [00:32<00:00,  1.65it/s]


In [1]:
import os
import time 
import re
from datetime import datetime

os.chdir('/compass/Shared/Users/bogdanov/vyzkumny_ukol/')
from pathlib import Path
import matplotlib.pyplot as plt
from cdb_extras import xarray_support as cdbxr   # načítání dat z databáze COMPASSu
from pyCDB import client
import pandas as pd
import matplotlib.pyplot as plt
import imgs_processing as imgs
import numpy as np
import xrscipy.signal as dsp
from tqdm import tqdm
cdb = client.CDBClient()

import process_data_for_alt_models as pdalt

path = Path('/compass/Shared/Users/bogdanov/vyzkumny_ukol')

In [3]:
#dirs where to save the csv files
shot_numbers = [17837]
variant = 'seidl_2023'
directories = {'h_alpha':'data/h_alpha_signal', 
                'mc':'data/mirnov_coil_signal', 
                'divlp':'data/langmuir_probe_signal'}
for shot in tqdm(shot_numbers):
    print('working on shot:', shot)
    # Load signals from CDB
    h_alpha_signal = cdb.get_signal(f"H_alpha/SPECTROMETRY_RAW:{shot}")
    mc_signal = cdb.get_signal(f"Mirnov_coil_A_theta_13_RAW/MAGNETICS_RAW:{shot}")
    divlp_signal = cdb.get_signal(f"DIVLPB01/STRATUS:{shot}")
    # Load labels from CDB
    t_ELM_start = cdb.get_signal(f"t_ELM_start/SYNTHETIC_DIAGNOSTICS:{shot}:{variant}")
    t_ELM_peak = cdb.get_signal(f"t_ELM_peak/SYNTHETIC_DIAGNOSTICS:{shot}:{variant}")
    #Calculate t_elm_end from t_ELM_peak and t_ELM_start (it should be symetric to t_ELM_peak)
    t_ELM_end = 2*t_ELM_peak - t_ELM_start
    #t_ELM_end = cdb.get_signal(f"t_ELM_end/SYNTHETIC_DIAGNOSTICS:{shot}:{variant}")
    t_H_mode_start = cdb.get_signal(f"t_H_mode_start/SYNTHETIC_DIAGNOSTICS:{shot}:{variant}")
    t_H_mode_end = cdb.get_signal(f"t_H_mode_end/SYNTHETIC_DIAGNOSTICS:{shot}:{variant}")
    #TODO:  To create a DataFrame with only one row, one needs to specify an index, 
    # so if plasma enters H-mode more than once during one shot index have to be passed. Thus crutch with try: except:
    try:
        len(t_ELM_start.data)
    except:
        t_ELM = pd.DataFrame({'start':t_ELM_start.data, 'end':t_ELM_end.data}, index=[0])
    else:
        t_ELM = pd.DataFrame({'start':t_ELM_start.data, 'end':t_ELM_end.data})
    try:
        len(t_H_mode_start.data)
    except:
        t_H_mode = pd.DataFrame({'start':t_H_mode_start.data, 'end':t_H_mode_end.data}, index=[0])
    else:
        t_H_mode = pd.DataFrame({'start':t_H_mode_start.data, 'end':t_H_mode_end.data})
    for signal, signal_name in zip([h_alpha_signal, mc_signal, divlp_signal], ['h_alpha', 'mc', 'divlp']):
        signal_df = pd.DataFrame({'time':signal.time_axis.data, signal_name:signal.data})
        # Set time as index
        signal_df = signal_df.set_index('time')
        ### Downsample data
        #First define desired frequency
        desired_frequency = 300 #in kHz
        time_resolution = 1/desired_frequency
        # Find how many rows do we need to skip to get the desired frequency 
        #(raw data have different frequencies for different shots)
        skip_rows = 0
        while signal_df.index[skip_rows] - signal_df.index[0]  < time_resolution:
            skip_rows += 1
        # Downsample
        signal_df = signal_df.iloc[::skip_rows]
        # Remove data with no plasma
        discharge_start, discharge_end = imgs.discharge_duration(shot, 4e4)
        signal_df = signal_df[np.logical_and(signal_df.index>discharge_start, signal_df.index<discharge_end)]
        # Create a column with mode labels. These are all L-mode by default.
        signal_df['mode'] = 'L-mode'
        for H_mode in t_H_mode.values:
            signal_df.loc[H_mode[0]:H_mode[1], 'mode'] = 'H-mode'
        for elm in t_ELM.values:
            signal_df.loc[elm[0]:elm[1], 'mode'] = 'ELM'
        # Save data
        signal_df.to_csv(f'{directories[signal_name]}/shot_{shot}.csv')

  0%|          | 0/1 [00:00<?, ?it/s]

working on shot: 17837


/compass/Shared/Common/IT/projects/user-libraries/python/cdb_extras/dev/cdb_extras/signal_plugins/core.py:76: UserWarning: cdb_extras.signal_plugins will be deprecated by cdb_extras.xarray_support and is not maintained anymore
  warnings.warn('cdb_extras.signal_plugins will be deprecated'


In [18]:

pdalt.process_data_for_alt_models([17837], variant='seidl_2023')

  0%|          | 0/1 [00:00<?, ?it/s]

working on shot: 17837


/compass/Shared/Common/IT/projects/user-libraries/python/cdb_extras/dev/cdb_extras/signal_plugins/core.py:76: UserWarning: cdb_extras.signal_plugins will be deprecated by cdb_extras.xarray_support and is not maintained anymore
  warnings.warn('cdb_extras.signal_plugins will be deprecated'


In [ ]:
shot_usage = pd.read_csv(f'{path}/data/shot_usage.csv')
shot_numbers = shot_usage['shot']

pdalt.process_data_for_multiple_mirnov_coils(shot_numbers, variant='seidl_2023')